In [1]:
import pickle
with open('../project/data/train_DATA.pickle', 'rb') as f:
    train_DATA = pickle.load(f)
with open('../project/data/test_DATA.pickle', 'rb') as f:
    test_DATA = pickle.load(f)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import concatenate,Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from sklearn.metrics import roc_curve
from keras.utils import np_utils
from tensorflow.keras.metrics import binary_accuracy
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa 
import librosa.display
import pylab
import cv2
import os

In [3]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self,imgfiles,labels,batch_size,target_size=(64,64),shuffle=False,scale=255,n_classes=1,n_channels=3):
        self.batch_size = batch_size
        self.dim        = target_size
        self.labels     = labels
        self.imgfiles   = imgfiles
        self.n_classes  = n_classes
        self.shuffle    = shuffle
        self.n_channels = n_channels
        self.scale      = scale
        self.c          = 0
        self.on_epoch_end() 

    def __len__(self):
        # returns the number of batches
        return int(np.floor(len(self.imgfiles) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.imgfiles))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img   = cv2.imread(self.imgfiles[ID])
            img   = cv2.resize(img,self.dim,interpolation = cv2.INTER_CUBIC)

            X[i,] = img / self.scale
            X[i,] = img 

            # Store class
            y[i] = self.labels[ID]

            self.c +=1
        return X, y #keras.utils.to_categorical(y, num_classes=self.n_classes)


class CustomPipeline(tf.keras.utils.Sequence):
    def __init__(self,data_x,data_y,batch_size=48,shuffle=False,n_classes=1):
        self.features   = data_x
        self.labels     = data_y
        self.batch_size = 48
        self.shuffle    = shuffle
        self.n_features = self.features.shape[1]
        self.n_classes  = 1
        self.on_epoch_end()

    def __len__(self):
        
        return int(np.floor(len(self.features) / self.batch_size))

    def __getitem__(self,index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.features))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
 
    def __data_generation(self,indexes):
        X = np.empty((self.batch_size, self.n_features))
        y = np.empty((self.batch_size), dtype=int)

        for i, ID in enumerate(indexes):
            X[i,] = self.features[ID]
            y[i,] = self.labels[ID]
        return X, y
    
class Generator(keras.utils.Sequence):
    """Wrapper of two generatos for the combined input model"""

    def __init__(self, X, Y, batch_size,target_size=(64,64)):
        self.genX = CustomDataset(X, Y, batch_size=batch_size,shuffle=False,target_size=target_size)

    def __len__(self):
        return self.genX.__len__()

    def __getitem__(self, index):
        X_batch, Y_batch = self.genX.__getitem__(index)
        return X_batch, Y_batch
    

In [31]:
def build_model():
    '''Function to build ensemble model'''

    inp1   = Input(shape=(64,64,3))
    lay1_  = Conv2D(64, (3, 3), strides=(2, 2))(inp1)
    lay2_  = AveragePooling2D((2, 2), strides=(2,2))(lay1_)
    lay3_  = BatchNormalization()(lay2_)
    lay4_  = Activation('relu')(lay3_)

    lay5_  = Conv2D(128, (3, 3), padding="same") (lay4_)
    lay6_  = AveragePooling2D((2, 2), strides=(2,2)) (lay5_)
    lay7_  = BatchNormalization()(lay6_)
    lay8_  = Activation('relu')(lay7_)

    lay9_  = Conv2D(128, (3, 3), padding="same") (lay8_)
    lay10_ = AveragePooling2D((2, 2), strides=(2,2)) (lay9_)
    lay11_ = BatchNormalization()(lay10_)
    lay12_ = Activation('relu')(lay11_)

    lay13_ = Flatten()(lay12_)
    lay14_ = Dense(256)(lay13_)
    lay15_ = Dropout(rate=0.2)(lay14_)
    lay16_ = Dense(256)(lay15_)
    lay17_ = Dropout(rate=0.2)(lay16_)

    output  = Dense(1, activation='sigmoid')(lay17_)
    MELMODEL   = Model(inputs=inp1, outputs=output)

    return MELMODEL

In [32]:
def ResNet(base_model='resnet50'):
    inputs = tf.keras.layers.Input(shape=(64,64,3))

    base_encoder = tf.keras.applications.ResNet50(include_top=False, weights=None, input_tensor=None,
                                                  input_shape=(64,64,3), pooling='avg')
    base_encoder.training = True
    h = base_encoder(inputs)

    x = tf.keras.layers.Dense(256)(h)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dense(256)(x)
    output  = Dense(1, activation='sigmoid')(h)

    return tf.keras.Model(inputs=inputs, outputs=output)

In [33]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [38]:
# Create directory to save models 

save_dir = '../project/saved_models/'

fold_var = 1
NUM= 5

VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
val_acc = 0


# Run each split
for RUN in range(NUM):
    with tf.device("/gpu:0"):
        
        # CREATE NEW MODEL
        model = build_model()
        Adam = tf.keras.optimizers.Adam(0.00001)
        
        # COMPILE NEW MODEL
        model.compile(loss='BinaryCrossentropy',optimizer=Adam,metrics=['accuracy'])

        
        imgs     = train_DATA[RUN]['MEL']
        labels    = train_DATA[RUN]['LABEL']

        test_imgs    = test_DATA[RUN]['MEL']
        test_labels  = test_DATA[RUN]['LABEL']      
        
        TRAIN      = Generator(imgs,labels,batch_size=5,target_size=(64,64))
        TEST          = Generator(test_imgs,test_labels,batch_size=5,target_size=(64,64))
        
        # CREATE CALLBACKS
        checkpointer = keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), monitor='val_accuracy',verbose=1, save_best_only=True)

        # FIT THE MODEL
        history = model.fit_generator(TRAIN,
            epochs=40,
            validation_data=TEST,
            callbacks=[checkpointer])

#         # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights("../project/saved_models/model_"+str(fold_var)+".h5")

        results = model.evaluate_generator(TEST)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDAITON_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1 

<ipython-input-38-02c6c5016293>:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(TRAIN,


Epoch 1/40
13/16 [=======================>......] - ETA: 0s - loss: 0.9173 - accuracy: 0.3846
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to ../project/saved_models/model_1.h5
16/16 [==============================] - 1s 31ms/step - loss: 0.9194 - accuracy: 0.4000 - val_loss: 0.7041 - val_accuracy: 0.5000
Epoch 2/40
16/16 [==============================] - ETA: 0s - loss: 0.8396 - accuracy: 0.4625
Epoch 00002: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 22ms/step - loss: 0.8396 - accuracy: 0.4625 - val_loss: 0.6986 - val_accuracy: 0.5000
Epoch 3/40
16/16 [==============================] - ETA: 0s - loss: 0.7599 - accuracy: 0.5500
Epoch 00003: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 22ms/step - loss: 0.7599 - accuracy: 0.5500 - val_loss: 0.7103 - val_accuracy: 0.5000
Epoch 4/40
13/16 [=======================>......] - ETA: 0s - loss: 0.7770 - accuracy: 0.4923
Epoch 00004: val

13/16 [=======================>......] - ETA: 0s - loss: 0.4832 - accuracy: 0.7538
Epoch 00030: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 21ms/step - loss: 0.4707 - accuracy: 0.7625 - val_loss: 0.8575 - val_accuracy: 0.4500
Epoch 31/40
16/16 [==============================] - ETA: 0s - loss: 0.5166 - accuracy: 0.7625
Epoch 00031: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 21ms/step - loss: 0.5166 - accuracy: 0.7625 - val_loss: 0.8592 - val_accuracy: 0.4500
Epoch 32/40
15/16 [===========================>..] - ETA: 0s - loss: 0.4283 - accuracy: 0.8267
Epoch 00032: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 23ms/step - loss: 0.4344 - accuracy: 0.8125 - val_loss: 0.8647 - val_accuracy: 0.4500
Epoch 33/40
16/16 [==============================] - ETA: 0s - loss: 0.4503 - accuracy: 0.8125
Epoch 00033: val_accuracy did not improve from 0.50000
16/16 [===============

<ipython-input-38-02c6c5016293>:46: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  results = model.evaluate_generator(TEST)


Epoch 1/40
16/16 [==============================] - ETA: 0s - loss: 0.8078 - accuracy: 0.4750
Epoch 00001: val_accuracy improved from -inf to 0.40000, saving model to ../project/saved_models/model_2.h5
16/16 [==============================] - 1s 32ms/step - loss: 0.8078 - accuracy: 0.4750 - val_loss: 0.7009 - val_accuracy: 0.4000
Epoch 2/40
13/16 [=======================>......] - ETA: 0s - loss: 0.6976 - accuracy: 0.6000
Epoch 00002: val_accuracy improved from 0.40000 to 0.50000, saving model to ../project/saved_models/model_2.h5
16/16 [==============================] - 0s 24ms/step - loss: 0.7334 - accuracy: 0.5625 - val_loss: 0.7020 - val_accuracy: 0.5000
Epoch 3/40
13/16 [=======================>......] - ETA: 0s - loss: 0.7376 - accuracy: 0.5231
Epoch 00003: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 21ms/step - loss: 0.7532 - accuracy: 0.5375 - val_loss: 0.7014 - val_accuracy: 0.5000
Epoch 4/40
13/16 [=======================>......] - ET

Epoch 30/40
13/16 [=======================>......] - ETA: 0s - loss: 0.5124 - accuracy: 0.8154
Epoch 00030: val_accuracy did not improve from 0.55000
16/16 [==============================] - 0s 21ms/step - loss: 0.4799 - accuracy: 0.8375 - val_loss: 0.7837 - val_accuracy: 0.5500
Epoch 31/40
13/16 [=======================>......] - ETA: 0s - loss: 0.5333 - accuracy: 0.7231
Epoch 00031: val_accuracy did not improve from 0.55000
16/16 [==============================] - 0s 22ms/step - loss: 0.5282 - accuracy: 0.7125 - val_loss: 0.7834 - val_accuracy: 0.5500
Epoch 32/40
15/16 [===========================>..] - ETA: 0s - loss: 0.4558 - accuracy: 0.8400
Epoch 00032: val_accuracy improved from 0.55000 to 0.60000, saving model to ../project/saved_models/model_2.h5
16/16 [==============================] - 0s 25ms/step - loss: 0.4693 - accuracy: 0.8250 - val_loss: 0.7893 - val_accuracy: 0.6000
Epoch 33/40
16/16 [==============================] - ETA: 0s - loss: 0.5136 - accuracy: 0.7625
Epoch 000

Epoch 19/40
16/16 [==============================] - ETA: 0s - loss: 0.6220 - accuracy: 0.6625
Epoch 00019: val_accuracy did not improve from 0.65000
16/16 [==============================] - 0s 22ms/step - loss: 0.6220 - accuracy: 0.6625 - val_loss: 0.6694 - val_accuracy: 0.6500
Epoch 20/40
16/16 [==============================] - ETA: 0s - loss: 0.6087 - accuracy: 0.7000
Epoch 00020: val_accuracy did not improve from 0.65000
16/16 [==============================] - 0s 23ms/step - loss: 0.6087 - accuracy: 0.7000 - val_loss: 0.6681 - val_accuracy: 0.6500
Epoch 21/40
13/16 [=======================>......] - ETA: 0s - loss: 0.5974 - accuracy: 0.7231
Epoch 00021: val_accuracy did not improve from 0.65000
16/16 [==============================] - 0s 21ms/step - loss: 0.5974 - accuracy: 0.7250 - val_loss: 0.6642 - val_accuracy: 0.6000
Epoch 22/40
16/16 [==============================] - ETA: 0s - loss: 0.5068 - accuracy: 0.7375
Epoch 00022: val_accuracy did not improve from 0.65000
16/16 [===

Epoch 8/40
13/16 [=======================>......] - ETA: 0s - loss: 0.7908 - accuracy: 0.5231
Epoch 00008: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 20ms/step - loss: 0.8273 - accuracy: 0.4875 - val_loss: 0.6836 - val_accuracy: 0.5000
Epoch 9/40
13/16 [=======================>......] - ETA: 0s - loss: 0.7121 - accuracy: 0.5846
Epoch 00009: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 20ms/step - loss: 0.7117 - accuracy: 0.5875 - val_loss: 0.6909 - val_accuracy: 0.5000
Epoch 10/40
13/16 [=======================>......] - ETA: 0s - loss: 0.7019 - accuracy: 0.5846
Epoch 00010: val_accuracy did not improve from 0.50000
16/16 [==============================] - 0s 22ms/step - loss: 0.6806 - accuracy: 0.6125 - val_loss: 0.6822 - val_accuracy: 0.5000
Epoch 11/40
13/16 [=======================>......] - ETA: 0s - loss: 0.7293 - accuracy: 0.5231
Epoch 00011: val_accuracy did not improve from 0.50000
16/16 [=====

Epoch 37/40
13/16 [=======================>......] - ETA: 0s - loss: 0.6027 - accuracy: 0.6308
Epoch 00037: val_accuracy did not improve from 0.70000
16/16 [==============================] - 0s 21ms/step - loss: 0.5693 - accuracy: 0.6750 - val_loss: 0.7344 - val_accuracy: 0.5000
Epoch 38/40
13/16 [=======================>......] - ETA: 0s - loss: 0.5476 - accuracy: 0.7231
Epoch 00038: val_accuracy did not improve from 0.70000
16/16 [==============================] - 0s 21ms/step - loss: 0.5376 - accuracy: 0.7000 - val_loss: 0.7320 - val_accuracy: 0.5000
Epoch 39/40
13/16 [=======================>......] - ETA: 0s - loss: 0.4944 - accuracy: 0.8308
Epoch 00039: val_accuracy did not improve from 0.70000
16/16 [==============================] - 0s 19ms/step - loss: 0.5056 - accuracy: 0.7875 - val_loss: 0.7362 - val_accuracy: 0.5500
Epoch 40/40
13/16 [=======================>......] - ETA: 0s - loss: 0.5371 - accuracy: 0.7692
Epoch 00040: val_accuracy did not improve from 0.70000
16/16 [===

Epoch 26/40
13/16 [=======================>......] - ETA: 0s - loss: 0.6896 - accuracy: 0.6769
Epoch 00026: val_accuracy did not improve from 0.75000
16/16 [==============================] - 0s 20ms/step - loss: 0.6673 - accuracy: 0.6875 - val_loss: 0.6285 - val_accuracy: 0.7500
Epoch 27/40
13/16 [=======================>......] - ETA: 0s - loss: 0.6875 - accuracy: 0.6000
Epoch 00027: val_accuracy did not improve from 0.75000
16/16 [==============================] - 0s 21ms/step - loss: 0.6563 - accuracy: 0.6375 - val_loss: 0.6182 - val_accuracy: 0.7500
Epoch 28/40
16/16 [==============================] - ETA: 0s - loss: 0.5801 - accuracy: 0.7250
Epoch 00028: val_accuracy did not improve from 0.75000
16/16 [==============================] - 0s 21ms/step - loss: 0.5801 - accuracy: 0.7250 - val_loss: 0.6039 - val_accuracy: 0.7500
Epoch 29/40
16/16 [==============================] - ETA: 0s - loss: 0.5510 - accuracy: 0.7250
Epoch 00029: val_accuracy did not improve from 0.75000
16/16 [===

In [39]:
VALIDATION_ACCURACY = np.asarray(VALIDATION_ACCURACY)
acc = np.mean(VALIDATION_ACCURACY, axis=0)

In [40]:
acc

0.65

In [41]:
VALIDATION_ACCURACY

array([0.5       , 0.60000002, 0.64999998, 0.69999999, 0.80000001])